In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('hexoskin_2bands_training_set_10_final.xlsx')
df

,A_exp_dur_sd,exp_diff_mean,A_exp_diff_mean,exp_dur_sd,duty_cycle_mean,A_RRV_RMSSD,RRV_RMSSD,ie_ratio_mean,A_duty_cycle_mean,insp_flow_sd,Task_Label,Participant,Classification
0,0.164364,0.127790,0.117188,0.156604,0.492097,404.771735,376.386712,0.971761,0.492399,1.445912,12.0a,10785,0
1,0.174588,0.201451,0.241071,0.152211,0.486992,392.384431,351.137516,0.950939,0.485336,1.034691,12.0b,10785,0
2,0.251690,0.199219,0.236719,0.213642,0.476697,459.428807,457.608476,0.913378,0.482721,1.890795,12.0c,10785,0
3,0.197705,0.285156,0.266276,0.213173,0.477750,450.569753,439.827676,0.918119,0.484587,1.316481,12.0d,10785,0
4,0.193377,0.159375,0.108073,0.213828,0.481671,516.192869,457.645154,0.936229,0.488034,0.963488,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,0.209238,0.203125,0.218316,0.200555,0.490259,357.971103,334.175635,0.967894,0.488741,2.209139,85,98586,0
2900,0.731201,1.195871,0.941894,1.135998,0.408652,1042.040237,1022.081981,0.977483,0.431877,12.234752,87,98586,1
2901,0.115003,0.106337,0.101562,0.117894,0.496388,153.331025,185.358320,0.987026,0.494804,2.990174,89,98586,0
2902,1.647685,2.145182,2.380208,1.447558,0.412197,3239.668637,3053.223554,0.748645,0.374552,14.419616,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[ 0.1643645,  0.1277902,  0.1171875, ...,  0.971761 ,  0.4923987,
          1.4459118],
        [ 0.1745884,  0.2014509,  0.2410714, ...,  0.9509391,  0.4853361,
          1.0346914],
        [ 0.2516897,  0.1992188,  0.2367188, ...,  0.9133779,  0.4827207,
          1.8907954],
        ...,
        [ 0.1150031,  0.1063368,  0.1015625, ...,  0.9870256,  0.4948042,
          2.9901739],
        [ 1.6476847,  2.1451823,  2.3802083, ...,  0.7486445,  0.3745517,
         14.419616 ],
        [ 0.1633064,  0.1235352,  0.1328125, ...,  0.926293 ,  0.4833117,
          2.0401398]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['A_exp_dur_sd',
  'exp_diff_mean',
  'A_exp_diff_mean',
  'exp_dur_sd',
  'duty_cycle_mean',
  'A_RRV_RMSSD',
  'RRV_RMSSD',
  'ie_ratio_mean',
  'A_duty_cycle_mean',
  'insp_flow_sd'],
 'participants': array([10785, 10785, 10785, ..., 98586, 98586, 98586]),
 'task': array(['12.0a', '12.0b', '12.0c', ..., 89, 91, 93], dtype=object)}

In [4]:
#running the nested cross-validation for the 2bandsRIP method using Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'penalty' : ['l1', 'l2'],
    'C' : (1, 4, 20),
    'solver' : ['liblinear','lbfgs']
}


lrc = LogisticRegression(random_state=random_seed)

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=lrc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

...

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

nested_accuracy_scores:    [0.93939394 0.84848485 0.63636364 0.84848485 0.98484848 0.95454545
 1.         0.87878788 0.95454545 0.92424242 0.92424242 0.66666667
 0.77272727 0.83333333 0.86363636 0.93939394 1.         0.75757576
 0.92424242 0.92424242 0.86363636 0.93939394 0.83333333 0.87878788
 0.87878788 0.90909091 0.93939394 0.96969697 0.83333333 0.6969697
 0.90909091 1.         0.84848485 0.96969697 0.84848485 0.93939394
 0.90909091 0.87878788 0.96969697 0.89393939 0.83333333 0.84848485
 0.71212121 0.93939394]
mean score:            0.8822
nested_precision_scores:    [0.94466403 0.86272727 0.76033058 0.85885167 0.98520085 0.95757576
 1.         0.87809917 0.95459141 0.92532151 0.92981093 0.68888889
 0.79824047 0.85098039 0.87471352 0.94213287 1.         0.75636364
 0.92398198 0.92981093 0.88797654 0.94466403 0.88571429 0.89818182
 0.8885851  0.92045455 0.94008264 0.97107438 0.83215369 0.69023569
 0.90909091 1.         0.85885167 0.96969697 0.84848485 0.94805195
 0.92045455 0.8806324

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.8333
Specificity: 1.0000
Fold 2:
Sensitivity: 0.6250
Specificity: 0.9762
Fold 3:
Sensitivity: 0.9167
Specificity: 0.4762
Fold 4:
Sensitivity: 0.8750
Specificity: 0.8333
Fold 5:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 6:
Sensitivity: 0.8750
Specificity: 1.0000
Fold 7:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 8:
Sensitivity: 0.7917
Specificity: 0.9286
Fold 9:
Sensitivity: 0.9167
Specificity: 0.9762
Fold 10:
Sensitivity: 0.9167
Specificity: 0.9286
Fold 11:
Sensitivity: 0.9583
Specificity: 0.9048
Fold 12:
Sensitivity: 0.6667
Specificity: 0.6667
Fold 13:
Sensitivity: 0.8333
Specificity: 0.7381
Fold 14:
Sensitivity: 0.5833
Specificity: 0.9762
Fold 15:
Sensitivity: 0.6667
Specificity: 0.9762
Fold 16:
Sensitivity: 0.9583
Specificity: 0.9286
Fold 17:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 18:
Sensitivity: 0.5000
Specificity: 0.9048
Fold 19:
Sensitivity: 0.8750
Specificity: 0.9524
Fold 20:
Sensitivity: 0.9583
Specificity: 0.9048
Fold 21:
Sensitivity: 0.9583


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.8822314049586777, SD=0.08736720404267594, 95% CI=(0.8564165202223102, 0.9080462896950452), Min=0.6363636363636364, Max=1.0
test_precision: Mean=0.8935524741301673, SD=0.07526812754546436, 95% CI=(0.871312572347809, 0.9157923759125255), Min=0.6888888888888889, Max=1.0
test_recall: Mean=0.8822314049586777, SD=0.08736720404267594, 95% CI=(0.8564165202223102, 0.9080462896950452), Min=0.6363636363636364, Max=1.0
test_f1_score: Mean=0.8709711520927482, SD=0.09490730736004484, 95% CI=(0.842928350139949, 0.8990139540455474), Min=0.6180555555555556, Max=1.0
test_roc_auc: Mean=0.9457972582972584, SD=0.07350744478050468, 95% CI=(0.9240775954664927, 0.967516921128024), Min=0.636904761904762, Max=1.0
sensitivity_scores: Mean=0.8314393939393938, SD=0.14791688379025048, 95% CI=(0.787733550912269, 0.8751452369665186), Min=0.3333333333333333, Max=1.0
specificity_scores: Mean=0.9112554112554112, SD=0.11531966265258346, 95% CI=(0.8771812545757597, 0.9453295679350627), Min=0.47619047

In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')


Fold 1 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 2 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 3 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 4 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 5 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 6 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 7 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 8 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 9 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 10 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 11 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 12 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 13 - Best hyperparamet

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: C - Most frequent value: 20 (Count: 40)
Hyperparameter: penalty - Most frequent value: l1 (Count: 44)
Hyperparameter: solver - Most frequent value: liblinear (Count: 44)
